# CO₂ Reduction vs Profitability Trade-off

This notebook computes and visualizes the trade-off function between CO₂ reduction and profitability by simulating tax rates up to 8000 ¥/tCO₂.


## Setup


In [ ]:
import pysd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import numpy as np

current_dir = Path.cwd()
BASE_DIR = current_dir.parent if current_dir.name == 'notebooks' else current_dir
os.chdir(BASE_DIR)

# Load the default parameters
with open(BASE_DIR / 'params.yaml', 'r') as file:
    base_params = yaml.safe_load(file)

model = pysd.load('src/model.py')

from src.utils.max_tax import calculate_max_viable_tax


## Calculate Tax Frontier


In [ ]:
tax_frontier = calculate_max_viable_tax(model, base_params)
print(f"Tax frontier: {tax_frontier} ¥/tCO₂")


## Baseline Simulation


In [ ]:
return_columns = [
    "cumulative_co2",
    "cumulative_profit",
    "viability_flag"
]

# Run baseline simulation
baseline_run = model.run(params=base_params, return_columns=return_columns)

co2_base = baseline_run["cumulative_co2"].iloc[-1]
profit_base = baseline_run["cumulative_profit"].iloc[-1]

print(f"Baseline CO₂: {co2_base:.2e} tCO₂")
print(f"Baseline Profit: {profit_base:.4f}")


## Simulate Tax Rates up to 8000


In [ ]:
# Vector of taxes to simulate. Using 30 points for now.
tax_grid = np.linspace(0, 6500, 40)

results_list = []

for tax in tax_grid:
    params = base_params.copy()
    params["carbon_tax_rate"] = tax
    sim_results = model.run(
        params=params,
        return_columns=return_columns
    )


    co2 = sim_results["cumulative_co2"].iloc[-1]
    profit = sim_results["cumulative_profit"].iloc[-1]

    viability_flag = sim_results["viability_flag"].iloc[-1]

    co2_reduction_pct = 100 * (1 - co2 / co2_base)
    profit_change_pct = 100 * ((profit - profit_base) / profit_base)

    results_list.append({
        "tax": tax,
        "co2": co2,
        "profit": profit,
        "co2_reduction_pct": co2_reduction_pct,
        "profit_change_pct": profit_change_pct,
        "viable": viability_flag
    })

results_df = pd.DataFrame(results_list)


## Run Exact Simulations for Specific Tax Rates


In [ ]:
# Run exact simulations for specific tax rates
current_tax = 289
exact_taxes = [current_tax, tax_frontier]

exact_results = {}

for tax in exact_taxes:
    params = base_params.copy()
    params["carbon_tax_rate"] = tax
    sim_results = model.run(
        params=params,
        return_columns=return_columns
    )
    
    co2 = sim_results["cumulative_co2"].iloc[-1]
    profit = sim_results["cumulative_profit"].iloc[-1]
    viability_flag = sim_results["viability_flag"].iloc[-1]
    
    co2_reduction_pct = 100 * (1 - co2 / co2_base)
    profit_change_pct = 100 * ((profit - profit_base) / profit_base)
    
    exact_results[tax] = {
        "tax": tax,
        "co2": co2,
        "profit": profit,
        "co2_reduction_pct": co2_reduction_pct,
        "profit_change_pct": profit_change_pct,
        "viable": viability_flag
    }
    
    print(f"Tax {tax} ¥/tCO₂: CO₂ reduction = {co2_reduction_pct:.2f}%, Profit change = {profit_change_pct:.2f}%")


## Plot Tradeoff

In [ ]:
fig, ax = plt.subplots()

# Get exact results for specific tax rates
current_tax = 289
current_result = exact_results[current_tax]
frontier_result = exact_results[tax_frontier]

# Plot all regular points from the grid
sc = ax.scatter(
    results_df["co2_reduction_pct"],
    results_df["profit_change_pct"],
    c=results_df["tax"],  # color by tax level
    s=30,  # regular size
)

# Plot current case point (289 ¥/tCO₂) with exact values
ax.scatter(
    current_result["co2_reduction_pct"],
    current_result["profit_change_pct"],
    c='white',
    marker='D',  # diamond marker
    s=80,  # larger size
    edgecolors='black',
    linewidths=1,
    label=f'Current rate ({current_tax} ¥/tCO₂)',
    zorder=5  # make sure it's on top
)

# Plot tax frontier point with exact values
ax.scatter(
    frontier_result["co2_reduction_pct"],
    frontier_result["profit_change_pct"],
    c='white',
    marker='*',
    s=200,  # larger size
    edgecolors='black',
    linewidths=1,
    label=f'Highest feasible rate ({tax_frontier:.0f} ¥/tCO₂)',
    zorder=5  # make sure it's on top
)

plt.title("CO₂ Reduction vs Profitability Trade-off Curve\n40 evenly spaced tax rates between 0 and 6500 ¥/tCO₂")

# ax.axhline(0, linestyle="--", color='gray', alpha=0.5)  # zero profit-change line
ax.set_xlabel("CO₂ reduction vs baseline (%)")
ax.set_ylabel("Profit change vs baseline (%)")
ax.grid(True, alpha=0.2)  # Add grid
ax.legend(loc='lower left')
plt.colorbar(sc, label="Carbon tax (¥/tCO₂)")

plt.savefig("figures/results/tradeoff.png", dpi=300)
plt.show()